# Implement a feedback loop to update the model periodically with fresh manufacturing data.

In [12]:
# Import the required libarary
from tensorflow.keras import datasets , layers , models
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.models import save_model,Sequential,load_model
import numpy as np
import random
import pickle

In [5]:
# Get the current directory
current_dir = os.getcwd()

# Get the parent directory (one level up)
current_dir = os.path.dirname(current_dir)

# Get the parent directory (one level up)
parent_dir = os.path.dirname(current_dir)

# Print the parent directory
print("Parent Directory:", parent_dir)

Parent Directory: C:\Users\sonu.a.jain\AppData\Local\miniconda3\Scripts\faultyFinding


## Add the data in feedbackloopdata folder 

In [ ]:
dataset_dir = parent_dir+'/datasets/feedbackLoopData' # dataset directory

In [ ]:
# Define parameters for preprocessing
batch_size = 32
image_size = (160, 160)

In [ ]:
#Load the data with the help of the tensorflow keras utils model :
dataset =  tf.keras.utils.image_dataset_from_directory(dataset_dir ,image_size=image_size)                                                                                 
class_name = dataset.class_names

## Split the dataset in to train and validation set

In [ ]:
val_batches = tf.data.experimental.cardinality(dataset)
print("%d"%val_batches)
validation_dataset = dataset.take(val_batches//4)
train_dataset = dataset.skip(val_batches//4)

In [ ]:
# SPlit the dataset into test and validation dataset

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
print("%d"%val_batches)

test_dataset = validation_dataset.take(val_batches//2)
validation_dataset = validation_dataset.skip(val_batches//2)

In [ ]:
## Autotune : In TensorFlow, tf.data.AUTOTUNE is a special constant that can be used when configuring input pipelines for better 
## performance. It's particularly useful when dealing with input data pipelines that involve I/O operations. 

In [ ]:
AUTOTUNE =  tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset =  validation_dataset.prefetch(buffer_size=AUTOTUNE)

## Laod the pretrained model

In [6]:
# Load the saved model
models = load_model(parent_dir+"/models/vgg16_model.keras")

In [13]:
## Load the history of pretrained model to identify the epoch count till trained

In [9]:
with open(parent_dir+'/models/vgg16_training_history.pkl', 'rb') as file:
    loaded_history = pickle.load(file)

## Train with new fresh data

In [ ]:
fine_tune_epoch = 5
initial_epoch=len(loaded_history.epoch)
totol_epoch =  fine_tune_epoch+intial_epochs
history = models.fit(train_dataset,epochs=totol_epoch,initial_epoch=history.epoch[-1],validation_data=validation_dataset)

In [ ]:
# Plot the accuracy of trained model
plt.plot(history.history['accuracy'],label= 'accuracy')
plt.plot(history.history['val_accuracy'],label= 'val_accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.title("vgg16_accuracy_plot")
plt.savefig(parent_dir+'\\visuals\\vgg16_finetune_accuracy_plot.png')

In [ ]:
# Plot the loss of trained model
plt.plot(history.history['loss'],label= 'loss')
plt.plot(history.history['val_loss'],label= 'val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.title("vgg16_loss_plot")
plt.savefig(parent_dir+'\\visuals\\vgg16_finetune_loss_plot.png')

In [ ]:
# Save the model (Keeping the file name same so that the model can always retain the history of past trained data)
save_model(models , parent_dir+"/models/vgg16_model.keras")

In [ ]:
# Save the model (Keeping the file name same so that the history can always retain the history of past trained data)
with open(parent_dir+'/models/vgg16_training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

## Model Evaluation

In [ ]:
loss , accuracy = models.evaluate(test_dataset)